In [3]:
import sys
!{sys.executable} -m pip install faiss-cpu


[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
import faiss



In [5]:
tennessee = pd.read_csv(r"C:\Users\jessi\Desktop\Proyecto Final Henry\Tennesse.csv")
georgia = pd.read_csv(r"C:\Users\jessi\Desktop\Proyecto Final Henry\Georgia.csv")
ohio = pd.read_csv(r"C:\Users\jessi\Desktop\Proyecto Final Henry\Ohio.csv")
california = pd.read_csv(r"C:\Users\jessi\Desktop\Proyecto Final Henry\California.csv")
florida = pd.read_csv(r"C:\Users\jessi\Desktop\Proyecto Final Henry\Florida.csv")
washington = pd.read_csv(r"C:\Users\jessi\Desktop\Proyecto Final Henry\Washington.csv")
colorado = pd.read_csv(r"C:\Users\jessi\Desktop\Proyecto Final Henry\Colorado.csv")
massachusetts = pd.read_csv(r"C:\Users\jessi\Desktop\Proyecto Final Henry\Massachusetts.csv")
kansas = pd.read_csv(r"C:\Users\jessi\Desktop\Proyecto Final Henry\Kansas.csv")
north_carolina = pd.read_csv(r"C:\Users\jessi\Desktop\Proyecto Final Henry\North_Carolina.csv")
missouri = pd.read_csv(r"C:\Users\jessi\Desktop\Proyecto Final Henry\Missouri.csv")
minnesota = pd.read_csv(r"C:\Users\jessi\Desktop\Proyecto Final Henry\Minnesota.csv")
pennsylvania = pd.read_csv(r"C:\Users\jessi\Desktop\Proyecto Final Henry\Pennsylvania.csv")
utah = pd.read_csv(r"C:\Users\jessi\Desktop\Proyecto Final Henry\Utah.csv")
illinois = pd.read_csv(r"C:\Users\jessi\Desktop\Proyecto Final Henry\Illinois.csv")
texas = pd.read_csv(r"C:\Users\jessi\Desktop\Proyecto Final Henry\Texas.csv")
oregon = pd.read_csv(r"C:\Users\jessi\Desktop\Proyecto Final Henry\Oregon.csv")
new_york = pd.read_csv(r"C:\Users\jessi\Desktop\Proyecto Final Henry\New_York.csv")

dataframes = [tennessee, georgia, ohio, california, florida, washington, colorado, massachusetts, kansas, north_carolina, missouri, minnesota, pennsylvania, utah, illinois, texas, oregon, new_york]

estados = pd.concat(dataframes, ignore_index=True)

In [6]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer



In [7]:
estados = estados.dropna()


In [57]:
estados['combined_features'] = estados['nombre_local'].astype(str) + ' ' + estados['ciudad'].astype(str).fillna('') + ' ' + estados['review_sentiment'].astype(str).fillna('')

# Muestra de datos para evitar problemas de memoria
#sample_size = 2000  # Aumentar el tamaño de la muestra si es necesario
#df_sampled = estados.sample(n=sample_size, random_state=42)

# Vectorización TF-IDF
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df_sampled['combined_features'])

# Cálculo de la similitud del coseno
cosine_sim_sampled = cosine_similarity(tfidf_matrix, tfidf_matrix)
cosine_sim_sampled_df = pd.DataFrame(cosine_sim_sampled, index=df_sampled['nombre_local'], columns=df_sampled['nombre_local'])

def recommend_restaurant(restaurante, ciudad):
    restaurante = restaurante.strip()  

    if restaurante not in cosine_sim_sampled_df.index:
        return f"Restaurant named '{restaurante}' not found in the dataset."

    restaurantes_ciudad = df_sampled[df_sampled['ciudad'].str.lower() == ciudad.lower()]

    # Verificar que hay restaurantes en la ciudad especificada
    if restaurantes_ciudad.empty:
        return f"No restaurants found in the city '{ciudad}'."

    sim_scores = cosine_sim_sampled_df.loc[restaurante, restaurantes_ciudad['nombre_local']]

    sim_scores = sim_scores.sort_values(ascending=False)
    top_similar_restaurants = sim_scores.index[sim_scores.index != restaurante]

    recommended_restaurants = restaurantes_ciudad[restaurantes_ciudad['nombre_local'].isin(top_similar_restaurants)][['nombre_local', 'avg_rating', 'direccion', 'ciudad']]
    recommended_restaurants = recommended_restaurants.drop_duplicates(subset=['nombre_local']).iloc[:5]


    return recommended_restaurants 

# Ejemplo de uso
resultado = recommend_restaurant("DELICIO Coal Fired Pizza", "Cincinnati")
print(resultado)

TypeError: DataFrame.sort_values() missing 1 required positional argument: 'by'

In [68]:


restaurantes_50_review = estados['nombre_local'].value_counts()
restaurantes_50_review = restaurantes_50_review[restaurantes_50_review > 50].index

states_filtred = estados[estados['nombre_local'].isin(restaurantes_50_review)]

states_filtred['combined_features'] = states_filtred['nombre_local'].astype(str) + ' ' + estados_filtrado['ciudad'].astype(str).fillna('') + ' ' + estados_filtrado['review_sentiment'].astype(str).fillna('')

# Muestra de datos para evitar problemas de memoria
sample_size = 2000 
df_sampled1 = states_filtred.sample(n=sample_size, random_state=42)

# Vectorización TF-IDF
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df_sampled1['combined_features'])

# Cálculo de la similitud del coseno
cosine_sim_sampled = cosine_similarity(tfidf_matrix, tfidf_matrix)
cosine_sim_sampled_df = pd.DataFrame(cosine_sim_sampled, index=df_sampled1['nombre_local'], columns=df_sampled['nombre_local'])

def recommend_restaurant(restaurante, ciudad):
    restaurante = restaurante.strip()  

    if restaurante not in cosine_sim_sampled_df.index:
        return f"Restaurant named '{restaurante}' not found in the dataset."

    # Filtrar los restaurantes de la misma ciudad
    restaurantes_city = df_sampled1[df_sampled1['ciudad'].str.lower() == ciudad.lower()]

    if restaurantes_city.empty:
        return f"No restaurants found in the city '{ciudad}'."

    sim_scores = cosine_sim_sampled_df.loc[restaurante, restaurantes_city['nombre_local']]

    if isinstance(sim_scores, pd.DataFrame):
        sim_scores = sim_scores.iloc[0]

    sim_scores = sim_scores.sort_values(ascending=False)
    top_similar_restaurants = sim_scores.index[sim_scores.index != restaurante]

    recommended_restaurants = restaurantes_city[restaurantes_city['nombre_local'].isin(top_similar_restaurants)][['nombre_local', 'avg_rating', 'direccion', 'ciudad']]
    recommended_restaurants = recommended_restaurants.drop_duplicates(subset=['nombre_local']).iloc[:5]

    num_recommendations = recommended_restaurants.shape[0]

    return recommended_restaurants 

resultado = recommend_restaurant("The Lambs Club", "New York")
print(resultado)


                    nombre_local  avg_rating          direccion    ciudad
380286               Shake Shack         4.4   154 East 86th St  New York
440452  Chopt Creative Salad Co.         4.0       11 W 42nd St  New York
369050                  Buddakan         4.5         75 9th Ave  New York
371870               Café Boulud         4.5       20 E 76th St  New York
367094                Sarabeth's         4.1  40 Central Park S  New York


C:\Users\jessi\AppData\Local\Temp\ipykernel_14992\218628403.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  states_filtred['combined_features'] = states_filtred['nombre_local'].astype(str) + ' ' + estados_filtrado['ciudad'].astype(str).fillna('') + ' ' + estados_filtrado['review_sentiment'].astype(str).fillna('')
